In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import torch
import pdb

from pathlib import Path
from torch import nn
from torch.nn import functional as F
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

from yelp.dataset import ProjectDataset

In [ ]:
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.contrib.handlers import ProgressBar

In [ ]:
def set_all_seed(seed, cuda):
  np.random.seed(seed)
  torch.manual_seed(seed)
  if cuda:
    torch.cuda.manual_seed(seed)

In [ ]:
path = Path('./data/yelp')
review_csv = path/'reviews_with_splits_lite.csv'
scratch = path/'scratch'
vectorizer_path = scratch/'vectorizer.json'

In [ ]:
# dataset = ProjectDataset.load_data_and_create_vectorizer(review_csv)
# dataset.save_vectorizer(vectorizer_path)

In [ ]:
dataset = ProjectDataset.load_data_and_vectorizer(review_csv, vectorizer_path)
vectorizer = dataset.get_vectorizer()

dataset.set_split('train')
train_dl = DataLoader(dataset, batch_size=128, shuffle=True, drop_last=True)
dataset.set_split('val')
val_dl = DataLoader(dataset, batch_size=128, shuffle=True, drop_last=True)

In [ ]:
class ReviewClassifier(nn.Module):
  def __init__(self, num_features):
    super(ReviewClassifier, self).__init__()
    self.fc1 = nn.Linear(in_features=num_features, out_features=1)
    
  def forward(self, x_in, apply_sigmoid=False):
    y_out = self.fc1(x_in).squeeze(1)
    if apply_sigmoid:
      y_out = torch.sigmoid(y_out)
    return y_out

In [ ]:
classifier = ReviewClassifier(num_features=len(vectorizer.review_vocab))
optimizer = optim.Adam(classifier.parameters(), lr=0.001)
loss_func = nn.BCEWithLogitsLoss()

In [ ]:
def bce_logits_wrapper(output):
    y_pred, y = output
    y_pred = (torch.sigmoid(y_pred) > 0.5).long()
    return y_pred, y

In [ ]:
trainer = create_supervised_trainer(classifier, optimizer, loss_func, device='cuda:3')
evaluator = create_supervised_evaluator(classifier, metrics=\
                                        {'accuracy':Accuracy(bce_logits_wrapper),\
                                         'bce': Loss(loss_func)}, device='cuda:3')

In [ ]:
pbar = ProgressBar(persist=True)
pbar.attach(trainer, output_transform=lambda x: {'loss': x})

In [ ]:
@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(engine):
  evaluator.run(train_dl)
  metrics = evaluator.state.metrics
  pbar.log_message(f"Training Results - Epoch: {engine.state.epoch}\
                    Avg accuracy: {metrics['accuracy']:0.2f}\
                    Avg loss: {metrics['bce']:0.2f}")

In [ ]:
trainer.run(train_dl, max_epochs=2)

In [ ]:
from ignite.contrib.handlers.param_scheduler import LRScheduler

In [ ]:
itr = iter(train_dl)

In [ ]:
x,y = next(itr)
y_pred = classifier(x)
loss = loss_func(y_pred, y)